In [ ]:
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
try:
    driver = webdriver.Chrome()
except:
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    driver = webdriver.Chrome(options=chrome_options)
    print("Running in headless mode.")
df2 = pd.DataFrame(columns=["url", "heading", "content"])

In [ ]:

count = 1  # Start counting from 1000 as first 1000 articles are already saved
df = pd.read_csv("siasat_topic_city.csv", skiprows=range(1, 1))  # Skip the first 1000 rows
not_processed = []

try:
    for link in df["url"]:
        driver.get(link)
        time.sleep(10)

        art = driver.find_elements(By.CSS_SELECTOR, 'div.entry-content')
        if len(art) != 1:
            print(f"Some problem with link: {link}")
            not_processed.append(link)
            continue

        headline = driver.find_elements(By.CSS_SELECTOR, 'div.inside-page-hero.grid-container.grid-parent')
        if len(headline) != 1:
            print(f"Some problem with link: {link}")
            not_processed.append(link)
            continue

        header = headline[0].find_elements(By.TAG_NAME, 'h1')
        if len(header) != 1:
            print(f"Some problem with link: {link}")
            not_processed.append(link)
            continue
        header_text = header[0].get_attribute("innerText")

        data_str = ""
        p_tag = art[0].find_elements(By.TAG_NAME, 'p')
        for j in range(len(p_tag)):
            data_str = data_str + p_tag[j].get_attribute("innerText") + "\n"
        data_str = data_str.strip()

        # Add the new data to the existing saved DataFrame
        df2.loc[len(df2)] = [link, header_text, data_str]

        if count % 100 == 0:
            print(count)
        count += 1

        if count % 1000 == 0:
            df2.to_csv(f"siasat_city_article_{count}.csv", index=False)
            print(f"Saved partial CSV at {count} articles")

except Exception as e:
    print(f"Error occurred: {e}")
    df2.to_csv(f"siasat_city_article_partial_{count}.csv", index=False)
    print(f"Saved data up to {count} articles due to an error")

# Final save to include all articles processed
df2.to_csv("siasat_city_all_articles.csv", index=False)
print("All articles saved to 'siasat_city_all_articles.csv'")